In [4]:
import sqlite3
from datetime import datetime

In [17]:
conn = sqlite3.connect(r'D:\sqlite_data\customer_db.db')
cursor = conn.cursor()

In [3]:
current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_date)

2024-10-24 22:14:35


In [15]:
def manage_customer_data(status, first_name, last_name=None, email=None, phone=None, address=None, customer_id=None):
    # connect to database
    conn = sqlite3.connect(r'D:\sqlite_data\customer_db.db')
    cursor = conn.cursor()

    # get date for create/modify date
    current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    try:
        # status = insert
        if status.lower() == 'insert' and customer_id is None:
            cursor.execute('SELECT MAX(CustomerID) FROM customers')
            result = cursor.fetchone()
            new_customer_id = (result[0] or 0) + 1
            cursor.execute('''
                INSERT INTO customers (CustomerID, FirstName, LastName, Email, Phone, Address, CreateDate, ModifyDate)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)            
            ''', (new_customer_id, first_name, last_name, email, phone, address, current_date, current_date)
            )
            print(f'Customer {first_name} inserted successfully')

        # status = update
        elif status.lower() == 'update' and customer_id is not None:
            cursor.execute('''
                UPDATE customers 
                SET FirstName = ?, LastName = ?, Email = ?, Phone = ?, Address = ?, ModifyDate = ?
                WHERE CustomerID = ?
            ''', (first_name, last_name, email, phone, address, current_date, customer_id))
            print(f'Customer ID {customer_id} updated successfully')
        
        # status = delete
        elif status.lower() == 'delete' and customer_id is not None:
            cursor.execute('''
                DELETE FROM customers
                WHERE CustomerID = ?
            ''', (customer_id,))
            print(f'Customer ID {customer_id} deleted successfully')

        else:
            print(f'Invalid status or missing customerID for update/delete')

        # commit
        conn.commit()

    except sqlite3.Error as e:
        print(f'An error occurred: {e}')
        conn.rollback() # rollback in case error

    finally:
        # close connection
        conn.close()


In [18]:
# Insert a new customer
manage_customer_data('insert', 'John', 'Smith', 'j.smith@email.com', '9999999999', 'Address1')

Customer John inserted successfully


In [21]:
cursor.execute('SELECT * FROM customers')
results = cursor.fetchall()
for row in results:
    print(row)

(1, 'John', 'Smith', 'j.smith@email.com', '9999999999', 'Address1', '2024-10-24 23:10:03', '2024-10-24 23:10:03')


In [22]:
# Update an existing
manage_customer_data('update', 'John', 'Smith', 'j.smith@email.com', '9999999999', '456 Oak St', customer_id=1)

Customer ID 1 updated successfully


In [23]:
cursor.execute('SELECT * FROM customers')
results = cursor.fetchall()
for row in results:
    print(row)

(1, 'John', 'Smith', 'j.smith@email.com', '9999999999', '456 Oak St', '2024-10-24 23:10:03', '2024-10-24 23:15:28')


In [16]:
# Delete customer
manage_customer_data('delete', None, customer_id=1)

Customer ID 1 deleted successfully
